In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from fastkde import KDE
from gaussianmixture import GaussianMixture
from tqdm import tqdm_notebook as tqdm
import scipy.stats
import h5py
import os
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Show KDE with confidence intervals univariate case

In [ ]:
# Parameters
seed = 0
ndatapoints = 500
confidence = 0.95
nrepeat = 1000  # Number of repeats for the bootstrap
npdf = 101
overwrite = False

In [ ]:
# Create object for generating data from a Gaussian mixture
xlim = [-3, 3]
gm = GaussianMixture([-1, 1], [0.5, 0.3])
(xpdf,), ypdf = gm.pdf(minx=[xlim[0]], maxx=[xlim[1]], n=npdf)

### Plug-in method

The uncertainty can be approximated by $\sqrt{\frac{\mu_K}{n h^d} p(x)}$ with:

- $mu_K$ is a constant that only depends on the chosen Kernel: $mu_K = \int K(x)^2 dx$ where the integral is taken over all $x$.
- $n$ represents the number of datapoints
- $h$ denotes the bandwidth
- $d$ denotes the dimension of $x$
- $p(x)$ is the real probability density at $x$

In reality, the uncertainty also depends on terms that are proportional with higher order terms of $h$. It is assumed that $h$ is sufficiently small such that these terms can be ignored.

The plug-in method simply replaces $p(x)$ with the estimated probability density function $\hat{p}(x)$, where $\hat{p}(x)$ is estimated using Kernel Density Estimation.

In [ ]:
# Use the plug-in method
np.random.seed(seed)
x = gm.generate_samples(ndatapoints)
kde = KDE(data=x)
kde.compute_bw()  # Compute the bandwidth using one-leave-out cross validation
bandwidth = kde.bw  # Store the bandwidth for later usage
print("Bandwidth: {:.5f}".format(bandwidth))
kde.compute_kde()
ypdf_estimated = kde.score_samples(xpdf)
low_plugin, up_plugin = kde.confidence_interval(xpdf, confidence=confidence)

### Bootstrap

With bootstrapping, we create $N$ artifical datasets by taking each time $n$ datapoints from the original dataset with replacement. In total, the real PDF $p(x)$ is $N$ times estimated. 

The first bootstrap method (bootstrap and plug-in approach) estimates the variance of the estimated probability at $x$ using the $N$ estimates. This variance is used to compose the confidence intervals.

The second bootstrap method (simply called Bootstrap) directly composes the confedence intervals by taking the appropriate percentiles of the estimated probabilities at each value of $x$.  

In [ ]:
# Use the bootstrap method for determining the confidence interval
np.random.seed(seed)
filename = os.path.join("hdf5", "pdfs_bootstrap.hdf5")
if overwrite or not os.path.exists(filename):
    pdfs_bootstrap = np.zeros((nrepeat, len(xpdf)))
    for i in tqdm(range(nrepeat)):
        kde = KDE(data=x[np.random.choice(len(x), size=len(x), replace=True)], bw=bandwidth)
        kde.compute_kde()
        pdfs_bootstrap[i] = kde.score_samples(xpdf)
        
    # Store the PDFs
    with h5py.File(filename, "w") as f:
        f.create_dataset("pdfs", data=pdfs_bootstrap)
else:
    with h5py.File(filename, "r") as f:
        pdfs_bootstrap = f["pdfs"][:]
        
# The first bootstrap method (bootstrap and plug-in approach)
std = np.std(pdfs_bootstrap, axis=0)
zvalue = scipy.stats.norm.ppf(confidence/2+0.5)
low_bootstrap1 = ypdf_estimated - zvalue*std  # "Bootstrap and plug-in approach"
up_bootstrap1 = ypdf_estimated + zvalue*std

# The second bootstrap method (simply called "bootstrap")
deviation = np.percentile(np.abs(pdfs_bootstrap - np.mean(pdfs_bootstrap, axis=0)), confidence*100, axis=0)
low_bootstrap2 = ypdf_estimated - deviation
up_bootstrap2 = ypdf_estimated + deviation

In [ ]:
# Perform the KDE many times to see the real uncertainty
np.random.seed(seed)
pdfs = np.zeros((nrepeat, len(xpdf)))
filename = os.path.join("hdf5", "pdfs.hdf5")
if overwrite or not os.path.exists(filename):
    for i in tqdm(range(nrepeat)):
        x = gm.generate_samples(ndatapoints)
        kde = KDE(data=x)
        kde.compute_bw(min_bw=0.05, max_bw=0.5)
        kde.compute_kde()
        pdfs[i] = kde.score_samples(xpdf)
        
    # Store the PDFs
    with h5py.File(filename, "w") as f:
        f.create_dataset("pdfs", data=pdfs)
else:
    with h5py.File(filename, "r") as f:
        pdfs = f["pdfs"][:]

low_real = np.percentile(pdfs, (1-confidence)*50, axis=0)
up_real = np.percentile(pdfs, (1+confidence)*50, axis=0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(16, 10))

# Plot the results
plt_pdf, = ax.plot(xpdf, ypdf)
plt_estimated, = ax.plot(xpdf, ypdf_estimated)
plt_plugin = ax.plot(np.array([xpdf, xpdf]).T, np.array([low_plugin, up_plugin]).T, '--', color='#ff8080')
plt_bootstrap1 = ax.plot(np.array([xpdf, xpdf]).T, np.array([low_bootstrap1, up_bootstrap1]).T, '--', color='#80ff80')
plt_bootstrap2 = ax.plot(np.array([xpdf, xpdf]).T, np.array([low_bootstrap2, up_bootstrap2]).T, '--', color='#8080ff')
plt_realconf = ax.fill_between(xpdf, low_real, up_real, facecolor=[.6, .6, .6], alpha=.5)
ax.legend([plt_pdf, plt_estimated, plt_plugin[0], plt_bootstrap1[0], plt_bootstrap2[0], plt_realconf], 
          ['Real', 'Estimated', '{:.0f}% Confidence (plug-in)'.format(confidence*100),
           '{:.0f}% Confidence (bootstrap and plug-in)'.format(confidence*100),
           '{:.0f}% Confidence (bootstrap)'.format(confidence*100),
           '{:.0f}% Confidence (real)'.format(confidence*100)])
ax.grid(True)
_ = ax.set_xlim(xlim)  # The "_ =" suppresses the output

# Show confidence interval multivariate case (2-d)

In [ ]:
# Parameters
seed = 0
ndatapoints = 500
confidence = 0.95
nrepeat = 1000  # Number of repeats for the bootstrap
factor_of_max = 0.1  # Control height of PDF for contour plot
npdf = 101
overwrite = False

In [ ]:
# Create object for generating data from a Gaussian mixture
x1lim, x2lim = [-3, 3], [-3, 3]
gm2 = GaussianMixture([[-1, -1], [1, 1]], [[[0.5, 0.25], [0.25, 0.4]], [[0.5, -0.25], [-0.25, 0.4]]])
(x1pdf, x2pdf), ypdf = gm2.pdf(minx=[x1lim[0], x2lim[0]], maxx=[x1lim[1], x2lim[1]], n=npdf)

In [ ]:
# Use the plug-in method
np.random.seed(seed)
x = gm2.generate_samples(ndatapoints)
kde = KDE(data=x)
kde.compute_bw()  # Compute the bandwidth using one-leave-out cross validation
bandwidth = kde.bw  # Store the bandwidth for later usage
print("Bandwidth: {:.5f}".format(bandwidth))
kde.compute_kde()
xpdf2 = np.transpose(np.array([x1pdf, x2pdf]), [1, 2, 0])
ypdf_estimated = kde.score_samples(xpdf2)
low_plugin2, up_plugin2 = kde.confidence_interval(xpdf2, confidence=confidence)

In [ ]:
# Use the bootstrap method for determining the confidence interval
np.random.seed(seed)
filename = os.path.join("hdf5", "pdfs_bootstrap2.hdf5")
pdfs_bootstrap2 = np.zeros((nrepeat, len(x1pdf), len(x2pdf)))

if overwrite or not os.path.exists(filename):
    for i in tqdm(range(nrepeat)):
        kde = KDE(data=x[np.random.choice(len(x), size=len(x), replace=True)], bw=bandwidth)
        kde.compute_kde()
        pdfs_bootstrap2[i] = kde.score_samples(xpdf2)

        # Store the PDFs
        with h5py.File(filename, "w") as f:
            f.create_dataset("pdfs", data=pdfs_bootstrap2)
else:
    with h5py.File(filename, "r") as f:
        pdfs_bootstrap2 = f["pdfs"][:]
        
std = np.std(pdfs_bootstrap2, axis=0)
zvalue = scipy.stats.norm.ppf(confidence/2+0.5)
low_bootstrap1_2 = ypdf_estimated - zvalue*std  # "Bootstrap and plug-in approach"
up_bootstrap1_2 = ypdf_estimated + zvalue*std
deviation = np.percentile(np.abs(pdfs_bootstrap2 - np.mean(pdfs_bootstrap2, axis=0)), confidence*100, axis=0)
low_bootstrap2_2 = ypdf_estimated - deviation
up_bootstrap2_2 = ypdf_estimated + deviation

In [ ]:
# Perform the KDE many times to see the real uncertainty
np.random.seed(seed)
filename = os.path.join("hdf5", "pdfs2.hdf5")
pdfs2 = np.zeros((nrepeat, len(x1pdf), len(x2pdf)))

if overwrite or not os.path.exists(filename):
    for i in tqdm(range(nrepeat)):
        x = gm2.generate_samples(ndatapoints)
        kde = KDE(data=x)
        kde.compute_bw(min_bw=0.05, max_bw=0.5)
        kde.compute_kde()
        pdfs2[i] = kde.score_samples(xpdf2)

        # Store the PDFs
        with h5py.File(filename, "w") as f:
            f.create_dataset("pdfs", data=pdfs2)
else:
    with h5py.File(filename, "r") as f:
        pdfs2 = f["pdfs"][:]
        
low_real2 = np.percentile(pdfs2, (1-confidence)*50, axis=0)
up_real2 = np.percentile(pdfs2, (1+confidence)*50, axis=0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(16, 10))

# Plot the result of the plug-in method
ycontour = np.max(ypdf)*factor_of_max
plt.contour(x1pdf, x2pdf, ypdf, ycontour, colors=plt_pdf.get_color())
plt.contour(x1pdf, x2pdf, ypdf_estimated, ycontour, colors=plt_estimated.get_color())
plt.contour(x1pdf, x2pdf, low_plugin2, ycontour, linestyles='dashed', colors=plt_plugin[0].get_color())
plt.contour(x1pdf, x2pdf, up_plugin2, ycontour, linestyles='dashed', colors=plt_plugin[0].get_color())
plt.contour(x1pdf, x2pdf, low_bootstrap1_2, ycontour, linestyles='dashed', colors=plt_bootstrap1[0].get_color())
plt.contour(x1pdf, x2pdf, up_bootstrap1_2, ycontour, linestyles='dashed', colors=plt_bootstrap1[0].get_color())
plt.contour(x1pdf, x2pdf, low_bootstrap2_2, ycontour, linestyles='dashed', colors=plt_bootstrap2[0].get_color())
plt.contour(x1pdf, x2pdf, up_bootstrap2_2, ycontour, linestyles='dashed', colors=plt_bootstrap2[0].get_color())
plt.contourf(x1pdf, x2pdf, up_real2, [ycontour, 10000], colors=plt_realconf.get_facecolor())
plt.contourf(x1pdf, x2pdf, low_real2, [ycontour, 10000], colors=[[1, 1, 1, 1]])
conf_patch = mpatches.Patch(color=plt_realconf.get_facecolor()[0])  # Because plt.contourf cannot be used as legend handle
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title("Contour plots at p = {:.3f} (p = pmax * {:.0f}%)".format(ycontour, factor_of_max*100))
plt.grid(True)
ax.legend([plt_pdf, plt_estimated, plt_plugin[0], plt_bootstrap1[0], plt_bootstrap2[0], conf_patch], 
          ['Real', 'Estimated', '{:.0f}% Confidence (plug-in)'.format(confidence*100),
           '{:.0f}% Confidence (bootstrap and plug-in)'.format(confidence*100),
           '{:.0f}% Confidence (bootstrap)'.format(confidence*100),
           '{:.0f}% Confidence (real)'.format(confidence*100)])

# Estimated MISE for increasing $n$

In [ ]:
# Parameters
nmax = 5000
nmin = 100
nstep = 10
seed = 1
overwrite = False

In [ ]:
# Generate datapoints
np.random.seed(seed)
x = gm.generate_samples(nmax)

In [ ]:
# Initialize KDE
kde = KDE(data=x)

In [ ]:
filename = os.path.join("hdf5", "mise.hdf5")
if overwrite or not os.path.exists(filename):
    nn = np.arange(nmin, nmax, nstep)
    bw = np.zeros_like(nn).astype(np.float)
    std = np.zeros_like(bw)
    for i, n in enumerate(tqdm(nn)):
        kde.set_n(n)
        kde.compute_bw()
        bw[i] = kde.bw
        std[i] = np.std(x[:n])
    mise = kde.muk / (nn * bw)
    
    # Write results to hdf5 file
    with h5py.File(filename, "w") as f:
        f.create_dataset("mise", data=mise)
        f.create_dataset("nn", data=nn)
        f.create_dataset("bw", data=bw)
else:
    with h5py.File(filename, "r") as f:
        nn = f["nn"][:]
        bw = f["bw"][:]
        mise = f["mise"][:]

In [ ]:
plt.loglog(nn, mise)
plt.xlabel("Number of samples")
plt.ylabel("MISE")
plt.grid(True)

# Show whether it is "better" to use multivariate distribution or not

In [ ]:
# Parameters
n = 100
seed = 1

In [ ]:
# Generate data
np.random.seed(seed)
x = gm.generate_samples(n)
x /= np.std(x)
y = gm.generate_samples(n)
y /= np.std(y)
xy = np.concatenate((x, y), axis=1)

In [ ]:
# Create three KDE (for x, y, and xy, respectively) and estimate MISE
kdex = KDE(data=x)
kdey = KDE(data=y)
kdexy = KDE(data=xy)
print("Description   Bandwidth   1LO Loglikelihood         MISE")
for kde, description in zip([kdex, kdey, kdexy], ['kdex', 'kdey', 'kdexy']):
    kde.compute_bw()
    print("{:>11s}   {:9.3f}   {:17.5e}   {:7.4e}".format(description, kde.bw, kde.score_leave_one_out(), 
                                                          kde.muk / (n * kde.bw**kde.d)))